In [22]:
%gui qt
import sys
from pathlib import Path
import urllib3

from dataclasses import dataclass
from scipy.spatial import ConvexHull

import numpy as np

import meshio
import zarr
import napari

sys.path.append("..")
import ply_zarr

from ome_zarr.utils import download

## Load data from IDR and download it

In [29]:
idr_data = "https://s3.embassy.ebi.ac.uk/idr/zarr/v0.1/6001247.zarr/"


idr_name = idr_data.rstrip('/').split("/")[-1]

local_path = Path("../data/") / idr_name
if not local_path.exists():
    download(idr_data, "../data/")


downloading...
   6001247.zarr
   6001247.zarr/labels
   6001247.zarr/labels/masks
to ../data/
[########################################] | 100% Completed |  4.3s
[########################################] | 100% Completed |  4.2s
[########################################] | 100% Completed |  3.6s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.2s
[########################################] | 100% Completed |  0.3s
[########################################] | 100% Completed |  0.8s


In [30]:
data = zarr.open(local_path.as_posix(), "a")

data.tree()

Tree(nodes=(Node(disabled=True, name='/', nodes=(Node(disabled=True, icon='table', name='0 (1, 2, 257, 210, 25…

In [31]:
labels = data['labels']["masks"][0][0]

lbls = np.unique(labels)

meshes = []
for idx in lbls[1:6]:
    _, x, y, z = (labels == idx).nonzero()
    points = np.vstack([x, y, z]).T
    hull = ConvexHull(points)
    triangles = hull.simplices
    color = (points * [0.1, 0.3, 0.2]).sum(axis=1)
    
    mesh = meshio.Mesh(
        points=points,
        cells=[("triangle", triangles),],
        point_data={"color": color})
    
    mesh.remove_orphaned_nodes()
    meshes.append(mesh)

In [6]:
# !rm -Rf {local_path / "meshes"}

In [32]:
mesh_grp = data.create_group('meshes')

In [33]:
for i, mesh in enumerate(meshes):
    grp = mesh_grp.create_group(i)
    ply_zarr.write(grp, mesh)


In [34]:
data.tree()

Tree(nodes=(Node(disabled=True, name='/', nodes=(Node(disabled=True, icon='table', name='0 (1, 2, 257, 210, 25…

In [39]:
mesh_grp[0].tree()

Tree(nodes=(Node(disabled=True, name='0', nodes=(Node(disabled=True, name='3', nodes=(Node(disabled=True, icon…

In [37]:
mesh_reloaded = ply_zarr.read(mesh_grp[0])

In [38]:
viewer = napari.Viewer()
viewer.add_image(data[0])

for mesh in meshes:
    viewer.add_surface(
        (mesh.points,
         mesh.cells[0].data,
         mesh.point_data["color"]),
        colormap="Blues", opacity=1
    )

surf = viewer.add_surface(
        (mesh_reloaded.points,
         mesh_reloaded.cells[0].data,
         mesh_reloaded.point_data["color"]),
        colormap="Reds", opacity=1
    )